In [ ]:

import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [ ]:
# Data Preprocessing

def preprocess_qos_data(data):
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data)
    return data_scaled, scaler

def preprocess_chat_data(chats, tokenizer, max_length=128):
    inputs = tokenizer(chats, padding=True, truncation=True, max_length=max_length, return_tensors="pt")
    return inputs


In [ ]:
# DNN for QoS Satisfaction Prediction

class QoSClassifier(nn.Module):
    def __init__(self, input_dim):
        super(QoSClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 2),  # Two classes: Satisfied, Unsatisfied
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.model(x)


In [ ]:
# BERT-based Sentiment Analysis

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

def sentiment_analysis(chats):
    inputs = preprocess_chat_data(chats, tokenizer)
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=1)
    return predictions


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Dynamic QoS Management

def adjust_qos(predictions, sentiment):
    if predictions == 1 or sentiment == 1:
        print("Increasing bandwidth and reducing latency.")
    else:
        print("Stable QoS parameters.")


In [ ]:
# Main Loop

def main_loop(qos_data, chat_data):
    qos_data, scaler = preprocess_qos_data(qos_data)
    qos_model = QoSClassifier(input_dim=qos_data.shape[1])

    for i, (metrics, chats) in enumerate(zip(qos_data, chat_data)):
        metrics = torch.tensor(metrics, dtype=torch.float32).unsqueeze(0)
        qos_prediction = qos_model(metrics)
        chat_sentiment = sentiment_analysis([chats])

        adjust_qos(qos_prediction.argmax().item(), chat_sentiment.item())
        print(f"Iteration {i+1}: QoS Prediction: {qos_prediction}, Chat Sentiment: {chat_sentiment}")


In [ ]:

# Example usage:
# Sample DataFrame for QoS metrics
qos_data = pd.DataFrame({
    'latency': [20, 30, 25, 40],
    'bitrate': [5000, 4500, 5200, 4800],
    'packet_loss': [0.1, 0.05, 0.07, 0.02]
})

# Sample chat messages
chat_data = [
    "This stream is amazing!",
    "Why is it lagging so much?",
    "The quality is really good today.",
    "Buffering again... not great."
]

# Run the main loop
main_loop(qos_data, chat_data)


Stable QoS parameters.
Iteration 1: QoS Prediction: tensor([[0.5905, 0.4095]], grad_fn=<SoftmaxBackward0>), Chat Sentiment: tensor([0])
Stable QoS parameters.
Iteration 2: QoS Prediction: tensor([[0.5457, 0.4543]], grad_fn=<SoftmaxBackward0>), Chat Sentiment: tensor([0])
Stable QoS parameters.
Iteration 3: QoS Prediction: tensor([[0.5376, 0.4624]], grad_fn=<SoftmaxBackward0>), Chat Sentiment: tensor([0])
Stable QoS parameters.
Iteration 4: QoS Prediction: tensor([[0.5284, 0.4716]], grad_fn=<SoftmaxBackward0>), Chat Sentiment: tensor([0])


In [ ]:
# High latency and packet loss scenario
qos_data = pd.DataFrame({
    'latency': [100, 150, 120, 130],
    'bitrate': [2000, 1800, 2100, 1900],
    'packet_loss': [0.15, 0.20, 0.18, 0.17]
})

# Chat data indicating user frustration
chat_data = [
    "The stream is lagging so much!",
    "Can't watch, it's buffering every minute.",
    "What's going on with the quality today?",
    "This is unwatchable!"
]

# Run the main loop
main_loop(qos_data, chat_data)


Increasing bandwidth and reducing latency.
Iteration 1: QoS Prediction: tensor([[0.4193, 0.5807]], grad_fn=<SoftmaxBackward0>), Chat Sentiment: tensor([0])
Stable QoS parameters.
Iteration 2: QoS Prediction: tensor([[0.5411, 0.4589]], grad_fn=<SoftmaxBackward0>), Chat Sentiment: tensor([0])
Increasing bandwidth and reducing latency.
Iteration 3: QoS Prediction: tensor([[0.4582, 0.5418]], grad_fn=<SoftmaxBackward0>), Chat Sentiment: tensor([0])
Increasing bandwidth and reducing latency.
Iteration 4: QoS Prediction: tensor([[0.4759, 0.5241]], grad_fn=<SoftmaxBackward0>), Chat Sentiment: tensor([0])


In [ ]:
# Smooth streaming metrics
qos_data = pd.DataFrame({
    'latency': [10, 12, 8, 15],
    'bitrate': [6000, 6200, 5900, 6100],
    'packet_loss': [0.01, 0.02, 0.00, 0.01]
})

# Chat data showing user satisfaction
chat_data = [
    "This stream is crystal clear!",
    "Loving the quality today!",
    "Perfect stream, no lag at all.",
    "Best quality I've seen so far!"
]

# Run the main loop
main_loop(qos_data, chat_data)


Stable QoS parameters.
Iteration 1: QoS Prediction: tensor([[0.5684, 0.4316]], grad_fn=<SoftmaxBackward0>), Chat Sentiment: tensor([0])
Stable QoS parameters.
Iteration 2: QoS Prediction: tensor([[0.5667, 0.4333]], grad_fn=<SoftmaxBackward0>), Chat Sentiment: tensor([0])
Stable QoS parameters.
Iteration 3: QoS Prediction: tensor([[0.6654, 0.3346]], grad_fn=<SoftmaxBackward0>), Chat Sentiment: tensor([0])
Stable QoS parameters.
Iteration 4: QoS Prediction: tensor([[0.5771, 0.4229]], grad_fn=<SoftmaxBackward0>), Chat Sentiment: tensor([0])


In [ ]:
# Mixed quality streaming
qos_data = pd.DataFrame({
    'latency': [25, 45, 20, 60],
    'bitrate': [5000, 4800, 5300, 4700],
    'packet_loss': [0.05, 0.10, 0.03, 0.12]
})

# Mixed user sentiment
chat_data = [
    "It's fine most of the time, but sometimes it lags.",
    "A bit choppy but still watchable.",
    "Smooth at first, but now it's buffering a lot.",
    "Why is it stuttering so much now?"
]

# Run the main loop
main_loop(qos_data, chat_data)


Stable QoS parameters.
Iteration 1: QoS Prediction: tensor([[0.5645, 0.4355]], grad_fn=<SoftmaxBackward0>), Chat Sentiment: tensor([0])
Stable QoS parameters.
Iteration 2: QoS Prediction: tensor([[0.5892, 0.4108]], grad_fn=<SoftmaxBackward0>), Chat Sentiment: tensor([0])
Stable QoS parameters.
Iteration 3: QoS Prediction: tensor([[0.5817, 0.4183]], grad_fn=<SoftmaxBackward0>), Chat Sentiment: tensor([0])
Stable QoS parameters.
Iteration 4: QoS Prediction: tensor([[0.6084, 0.3916]], grad_fn=<SoftmaxBackward0>), Chat Sentiment: tensor([0])


In [ ]:
# Idle or very low data streams
qos_data = pd.DataFrame({
    'latency': [5, 5, 5, 5],
    'bitrate': [1000, 1000, 1000, 1000],
    'packet_loss': [0.00, 0.00, 0.00, 0.00]
})

# Neutral chat messages
chat_data = [
    "Stream is okay.",
    "Nothing much happening right now.",
    "Just waiting for the game to start.",
    "Chat is kinda quiet today."
]

# Run the main loop
main_loop(qos_data, chat_data)


Stable QoS parameters.
Iteration 1: QoS Prediction: tensor([[0.5876, 0.4124]], grad_fn=<SoftmaxBackward0>), Chat Sentiment: tensor([0])
Stable QoS parameters.
Iteration 2: QoS Prediction: tensor([[0.5876, 0.4124]], grad_fn=<SoftmaxBackward0>), Chat Sentiment: tensor([0])
Stable QoS parameters.
Iteration 3: QoS Prediction: tensor([[0.5876, 0.4124]], grad_fn=<SoftmaxBackward0>), Chat Sentiment: tensor([0])
Stable QoS parameters.
Iteration 4: QoS Prediction: tensor([[0.5876, 0.4124]], grad_fn=<SoftmaxBackward0>), Chat Sentiment: tensor([0])


In [ ]:
# High bitrate for HD or 4K streaming
qos_data = pd.DataFrame({
    'latency': [10, 8, 12, 9],
    'bitrate': [8000, 8200, 7900, 8100],
    'packet_loss': [0.02, 0.01, 0.01, 0.00]
})

# Positive feedback in the chat
chat_data = [
    "The stream is so smooth!",
    "I love the 4K quality, it's amazing.",
    "Great job with the stream, perfect clarity.",
    "This is e-sports quality for sure!"
]

# Run the main loop
main_loop(qos_data, chat_data)


Increasing bandwidth and reducing latency.
Iteration 1: QoS Prediction: tensor([[0.4723, 0.5277]], grad_fn=<SoftmaxBackward0>), Chat Sentiment: tensor([0])
Increasing bandwidth and reducing latency.
Iteration 2: QoS Prediction: tensor([[0.4613, 0.5387]], grad_fn=<SoftmaxBackward0>), Chat Sentiment: tensor([0])
Stable QoS parameters.
Iteration 3: QoS Prediction: tensor([[0.5116, 0.4884]], grad_fn=<SoftmaxBackward0>), Chat Sentiment: tensor([0])
Increasing bandwidth and reducing latency.
Iteration 4: QoS Prediction: tensor([[0.4889, 0.5111]], grad_fn=<SoftmaxBackward0>), Chat Sentiment: tensor([0])
